# Task Index 維度驗證

## 目的
確認官方環境 `info['task_index']` 的真實維度，解決 83 vs 87 維度差異問題。

## 背景
- JAX preprocessor 當前輸出 **83 維**
- 官方 `main.py` 使用 `n_features=87`
- 假設：`info['task_index']` 可能是 **7 維**（非 3 維 one-hot）

## 重要
- 必須使用 `SAIClient.make_env()` 創建環境（不是 `gym.make()`）
- 環境變數 `MUJOCO_GL=disabled` 必須在最開頭設置

In [ ]:
# Cell 1: 環境變數設置（必須最先執行！）
import os
os.environ["MUJOCO_GL"] = "disabled"
os.environ["PYOPENGL_PLATFORM"] = "egl"
print("環境變數已設置 ✅")

In [ ]:
# Cell 2: 依賴檢查
import sys
print(f"Python: {sys.version}")

# 檢查 NumPy
try:
    import numpy as np
    print(f"NumPy: {np.__version__} {'✅' if np.__version__.startswith('1.') else '⚠️ NumPy 2.x'}")
except ImportError:
    print("❌ NumPy not installed")

# 檢查 sai-rl（關鍵！）
try:
    import sai_rl
    print(f"sai-rl: ✅")
except ImportError:
    print("❌ sai-rl 未安裝！請更新 requirements.txt 並重啟 cluster")

# 檢查 sai-mujoco
try:
    import sai_mujoco
    print(f"sai-mujoco: ✅")
except ImportError:
    print("❌ sai-mujoco not installed")

In [ ]:
# Cell 3: 使用 SAIClient 測試官方環境
from sai_rl import SAIClient
import numpy as np

# 官方競賽 ID
comp_id = "lower-t1-penalty-kick-goalie"

print(f"Creating environment with comp_id: {comp_id}")
sai = SAIClient(comp_id=comp_id)
env = sai.make_env()

obs, info = env.reset()

print(f"\n{'='*60}")
print(f"環境創建成功！")
print(f"{'='*60}")
print(f"\nRaw obs shape: {np.array(obs).shape}")

# 關鍵檢查：task_index 維度
if 'task_index' in info:
    task_index = np.array(info['task_index'])
    print(f"\n🎯 task_index:")
    print(f"   shape: {task_index.shape}")
    print(f"   dtype: {task_index.dtype}")
    print(f"   value: {task_index}")
else:
    print("\n⚠️ task_index NOT FOUND in info dict")
    print(f"Available keys: {list(info.keys())}")

In [ ]:
# Cell 4: 詳細列出 info dict 所有欄位維度
print("Info dict 欄位維度：")
print(f"{'='*70}")
print(f"{'Key':40s} | {'Shape':15s} | Dims")
print(f"{'-'*70}")

total_dims = 0
for key, value in sorted(info.items()):
    arr = np.array(value)
    dims = arr.size
    total_dims += dims
    print(f"{key:40s} | {str(arr.shape):15s} | {dims}")

print(f"{'='*70}")
print(f"Total info dimensions: {total_dims}")
print(f"Raw obs dimensions: {np.array(obs).size}")

In [ ]:
# Cell 5: 使用官方 Preprocessor 驗證最終維度
import sys
import os

# 添加 booster_soccer_showdown 到路徑
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, project_root)

try:
    from booster_soccer_showdown.training_scripts.main import Preprocessor
    
    preprocessor = Preprocessor()
    processed_obs = preprocessor.modify_state(obs, info)
    
    print(f"\n{'='*60}")
    print(f"官方 Preprocessor 輸出：")
    print(f"{'='*60}")
    print(f"Input obs shape:  {np.array(obs).shape}")
    print(f"Output obs shape: {processed_obs.shape}")
    print(f"\n🎯 最終維度: {processed_obs.shape[-1]}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\n嘗試手動計算維度...")
    
    # 根據 main.py 的 hstack 順序手動計算
    components = [
        ("robot_qpos", 12),
        ("robot_qvel", 12),
        ("project_gravity", 3),
        ("robot_gyro", 3),
        ("robot_accelerometer", 3),
        ("robot_velocimeter", 3),
        ("goal_team_0_rel_robot", 3),
        ("goal_team_1_rel_robot", 3),
        ("goal_team_0_rel_ball", 3),
        ("goal_team_1_rel_ball", 3),
        ("ball_xpos_rel_robot", 3),
        ("ball_velp_rel_robot", 3),
        ("ball_velr_rel_robot", 3),
        ("player_team", 2),
        ("goalkeeper_team_0_xpos_rel_robot", 3),
        ("goalkeeper_team_0_velp_rel_robot", 3),
        ("goalkeeper_team_1_xpos_rel_robot", 3),
        ("goalkeeper_team_1_velp_rel_robot", 3),
        ("target_xpos_rel_robot", 3),
        ("target_velp_rel_robot", 3),
        ("defender_xpos", 3),
        ("task_index", np.array(info.get('task_index', [])).size),
    ]
    
    total = sum(d for _, d in components)
    print(f"\n計算結果: {total} 維")

In [ ]:
# Cell 6: 清理
env.close()
print("環境已關閉 ✅")

## 結論

運行此 notebook 後，請確認：

1. `task_index` 維度是 **3** 還是 **7**？
2. 官方 Preprocessor 的最終輸出是 **83** 還是 **87** 維？
3. 如果是 83 維，則 `main.py` 中的 `n_features=87` 是錯誤的硬編碼值

### 下一步行動
- 如果確認是 83 維：更新 `preprocessor_jax.py` 註解，繼續開發
- 如果確認是 87 維：找出缺少的 4 維來源